In [11]:
import plotly.graph_objects as go

from datetime import datetime
import sys
import os


In [12]:
# Ralative imports
# Add the src directory to the Python path
sys.path.insert(0, '../src')

from botcoin.data.data_fetcher import HistoricalDataManager, generate_price_stream

In [13]:
hdm = HistoricalDataManager(ticker="AAPL")

# Create a naive datetime
start   = datetime(year=2025, month=4, day=15, hour=0, minute=0, second=0)
end     = datetime(year=2025, month=4, day=16, hour=0, minute=0, second=0)

start   = hdm.tz.localize(start)
end     = hdm.tz.localize(end)

In [14]:
hdm._check_data_in_local(start, end)

False

In [15]:
df = hdm.get_data(start=start, end=end)

2025-04-20 00:25:37,340 - HistoricalDataManager - INFO: Fetching data from Yahoo Finance.
2025-04-20 00:25:37,340 - HistoricalDataManager - INFO: Fetching data from 2025-04-15 00:00:00-04:00 to 2025-04-16 00:00:00-04:00
2025-04-20 00:25:37,355 - HistoricalDataManager - INFO: Data merged and saved to data\AAPL_1min_data.parquet


In [16]:
df.head()

,Close,High,Low,Open,Volume
Datetime,,,,,
2025-04-15 09:30:00-04:00,202.694595,202.869995,201.789993,201.854996,2412796
2025-04-15 09:31:00-04:00,202.744995,202.750000,202.160004,202.649994,312062
2025-04-15 09:32:00-04:00,202.294998,202.800003,202.070007,202.786804,263184
2025-04-15 09:33:00-04:00,202.339996,203.240005,202.309998,202.309998,399844
2025-04-15 09:34:00-04:00,202.547302,202.850006,202.033203,202.320007,296751


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 379 entries, 2025-04-15 09:30:00-04:00 to 2025-04-15 15:59:00-04:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   379 non-null    float64
 1   High    379 non-null    float64
 2   Low     379 non-null    float64
 3   Open    379 non-null    float64
 4   Volume  379 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 17.8 KB


In [18]:
prices = generate_price_stream(df, candle_duration='1min', avg_freq_per_minute=10)

In [19]:
def plot_candles_with_price_stream(ohlc_df, price_stream_df):
    """
    Plot OHLC candlesticks and overlay price stream using Plotly.
    
    Args:
        ohlc_df (pd.DataFrame): Must have DatetimeIndex and columns ['Open', 'High', 'Low', 'Close'].
        price_stream_df (pd.DataFrame): Must have DatetimeIndex and column ['price'].
    """
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=ohlc_df.index,
        open=ohlc_df['Open'],
        high=ohlc_df['High'],
        low=ohlc_df['Low'],
        close=ohlc_df['Close'],
        name='Candles',
        increasing_line_color='green',
        decreasing_line_color='red'
    ))

    # Simulated price points as a line
    fig.add_trace(go.Scatter(
        x=price_stream_df.index,
        y=price_stream_df['price'],
        mode='lines+markers',
        name='Simulated Prices',
        line=dict(color='blue', width=1),
        marker=dict(size=4)
    ))

    # Styling
    fig.update_layout(
        title="OHLC Candles with Simulated Tick Data",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        height=600,
        width=1000
    )

    fig.show()

In [20]:
plot_candles_with_price_stream(df, prices)